In [1]:
from PIL import Image
from io import BytesIO
from bs4 import BeautifulSoup
from base64 import b64decode
import re

In [2]:
def gif_from_in(notebook, nr, filename):
    def b64image(url):
        prefix = "data:image/png;base64,"
        assert url[:len(prefix)] == prefix
        img_bytes = b64decode(url[len(prefix):])
        return Image.open(BytesIO(img_bytes))

    def make_soup(notebook):
        with open(notebook) as html_file:
            soup = BeautifulSoup(html_file, "html5lib")
            return soup

    def find_out_for_in(nr, soup):
        code_cells = soup.findAll(class_="code_cell")
        for code_cell in code_cells:
            input_prompt = code_cell.find(class_="input_prompt")
            input_nr = int(re.search(r"\[(\d*)\]", input_prompt.text).group(1))
            if input_nr == nr:
                break
        else:
            return None

        return code_cell.find(class_="output")
    
    def images_from_out(out):
        png_url64s = []
        for output_area in out.find_all(class_="output_area"):
            png_divs = output_area.find_all(class_="output_png")
            for png_div in png_divs:
                for img in png_div.find_all("img"):
                    png_url64s.append(img.attrs["src"])

        images = [b64image(png_url64) for png_url64 in png_url64s]
        return images

    def save_to_gif(filename, images):
        images[0].save(filename, save_all=True, append_images=images[1:], duration=50, loop=2)
    
    soup = make_soup(notebook)
    out = find_out_for_in(nr, soup)
    images = images_from_out(out)
    save_to_gif(filename, images)

In [3]:
gif_from_in("./dlnd_face_generation.html", 13, "faces.gif")

In [4]:
gif_from_in("./dlnd_face_generation.html", 12, "digits.gif")